In [1]:
import grpc

# add a folder to the system path
import sys
sys.path.append("../protos")
import pipeline_pb2 as opencv_pb2
import pipeline_pb2_grpc as opencv_pb2_grpc
from aux import wrap_value, unwrap_value
import json
import numpy as np
import io

def bytes_to_np(b: bytes) -> np.ndarray:
    return np.load(io.BytesIO(b))


In [94]:
#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'
# List of file paths (example paths — replace with your actual ones)
file_paths = [
    #'00.jpg',
    '01.jpg',
]

#list all files in a certain directory
import os
directory = "/home/manuelf/mast3r/images_in/jpc_walk1"
file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# List to hold the binary data of each image
image_byte_list = []
for path in file_paths:
    # Open the file in binary read mode ('rb') and read its entire content
    with open(path, 'rb') as f:
        image_bytes = f.read()
        image_byte_list.append(image_bytes)
        #print(f"Read {path}: {len(image_bytes) / (1024 * 1024):.2f} MB")

in_json = {
    "opencv": {
        "parameters": {
            "feature_extractor": "SIFT",
            "max_keypoints": 2000, #if too large it overflows gRPC limit
            "ratio_thresh": 0.75
        }
    }
}

in_json = {
    "opencv": {
        "parameters": {
            "ssim_thresh": 0.90,
            "blur_kernel": 5,
            "motion_thresh": 30
        }
    }
}

idx = 0
for im in image_byte_list[::5]:
    request = opencv_pb2.Envelope(config_json = json.dumps(in_json),
                                  data = {"images": wrap_value([im])})

    #Para imagens muito grandes buffer grande
    channel_opt = [('grpc.max_send_message_length',-1), 
                   ('grp.max_receive_message_length',-1),
                   ('grpc.max_message_length', -1)]

    channel=grpc.insecure_channel(target,options=channel_opt)
    estimator_stub = opencv_pb2_grpc.PipelineServiceStub(channel)

    #response = estimator_stub.Process(request)
    response = estimator_stub.similarity_check(request)
    if (json.loads(response.config_json)["changed"]):
        print(f"Image {idx} changed")

    idx += 1


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8061: Failed to connect to remote host: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:8061: Failed to connect to remote host: connect: Connection refused (111)", grpc_status:14}"
>

In [29]:

kpts = bytes_to_np(unwrap_value(response.data["keypoints"]))
#descs = bytes_to_np(unwrap_value(response.data["descriptors"]))
try:
    inliersA = bytes_to_np(unwrap_value(response.data["matches_inliers_a"]))
    inliersB = bytes_to_np(unwrap_value(response.data["matches_inliers_b"]))

    if inliersA.shape == inliersB.shape and inliersA.shape[0] > 0:
        print(f"Number of inliers: {inliersA.shape[0]}")
        

except:
    print("No inliers")

print(response.config_json)

No inliers
{"status": "success", "runtime": 0.08534836769104004, "timestamp": 1759758729.4174473}


In [105]:
kpts.shape

(3, 501, 2)